In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Usado para o método split: é possível fazer um split manualmente, mas train_test_split é mais eficiente e há estratificação já implementada.
from sklearn.model_selection import train_test_split

In [2]:
# Realiza alguns imports para a especificação de tipos
from typing import Self, Optional, Union, List

In [109]:
class Dataset:
    def __init__( self, data: pd.DataFrame, *, label_column : int = -1, column_names : Optional[list[str]] = None, _label_categories : dict = None ) -> None:
        """
        Inicializa a classe Dataset.

        Args:
            data: DataFrame com os dados do conjunto de dados.
            label_column: Posição da coluna de rótulo (por padrão é -1, última coluna).
            column_names: Lista opcional com os nomes das colunas.
            _label_categories : Metadado passado entre instâncias, contém o mapeamento inteiro/texto dos rótulos.
        """

        self.data = data                            # Guarda o DataFrame do conjunto de dados
        self.label_column = label_column            # Armazena o índice para a coluna de rótulo
        self.column_names = column_names            # Armazena a lista com os rótulos das colunas do DataFrame
        self._label_categories = _label_categories  # Correspondência inteiro: texto da coluna de rótulo

        # Se for especificado, define o nome das colunas
        if self.column_names is not None:
            self._set_column_names()

    def _set_column_names( self ) -> None:
        """
        Atualiza os rótulos das colunas do DataFrame, conforme o valor no atributo column_names.
        """

        self.data.columns = self.column_names

    def shuffle( self, random_state: Optional[int] = None ) -> Self:
        """
        Retorna uma nova instância da classe com os dados embaralhados.

        Args:
            random_state: Semente opcional para reprodução de resultados.
        
        Returns:
            Uma nova instância do Dataset com os dados embaralhados.
        """

        # Obtém um DataFrame embaralhado e com os índices resetados
        data = self.data.sample(frac=1, random_state=random_state).reset_index( drop=True )

        # Retorna uma nova instância da classe
        return self.__class__( data, label_column = self.label_column, column_names = self.column_names, _label_categories = self._label_categories )
    
    def split( self, train_size: int = 0.8, *, random_state: Optional[int] = None, stratify: bool = False ) -> tuple[Self, Self]:
        """
        Divide o Dataset em conjunto de treinamento e de conjunto de teste.

        Args:
            train_size: Proporção do conjunto de treinamento (0-1).
            random_state: Semente opcional para reprodução de resultados.
            stratify: Se for True, manterá a proporção entre as classes
        
        Returns:
            Tupla( train_set, test_set )
        """

        # Verifica se stratify foi setado
        _stratify = self.y if stratify == True else None

        # Separa o conjunto de dados em dois
        train, test = train_test_split(
            self.data,
            train_size = train_size,
            random_state = random_state,
            stratify = _stratify
        )

        # Retorna as instâncias para Datasets dos conjuntos separados
        return (
            self.__class__( train, label_column = self.label_column, column_names = self.column_names, _label_categories = self._label_categories ),
            self.__class__( test, label_column = self.label_column, column_names = self.column_names, _label_categories = self._label_categories )
        )
    
    def remove_features( self, features_to_remove: List[ Union[int, str] ] ) -> Self:
        """ Remove features do conjunto de dados.
        
        Args:
            features_to_remove: Lista de índices (inteiros) ou de nomes das colunas a serem removidas.
        
        Returns:
            Um novo Dataset com as features restantes.

        Raises:
            ValueError: Formatos ou valores inválidos foram especificados em features_to_remove
        """

        # Realiza uma cópia do DataFrame (garante imutabilidade)
        new_data = self.data.copy( deep = True )
        m, n = new_data.shape

        # Índices a serem removidos
        cols_to_drop = set()

        # Percorre a lista especificada de atributos a serem removidos
        for feature in features_to_remove:
            # O atributo foi especificado como índice
            if isinstance(feature, int):
                # Trata índices negativos
                idx = feature if feature >= 0 else (feature + n)
                
                # Verifica se o índice está fora do intervalo permitido
                if (idx < 0) or (idx >= n):
                    raise ValueError(f"Índice {feature} fora do intervalo permitido.")
                
                cols_to_drop.add( idx )
            
            # O atributo foi especificado como string
            elif isinstance(feature, str):
                # Verifica se o atributo column_names está declarado
                if not self.column_names:
                    raise ValueError("Nomes de colunas não definidos.")
                
                # Recupera o índice do valor especificado
                try:
                    idx = self.column_names.index(feature)
                
                except ValueError:
                    raise ValueError(f"Coluna '{feature}' não encontrada.")
                
                cols_to_drop.add(idx)
            
            else:
                raise ValueError("Os elementos especificador devem ser str ou int")
        
        # Verifica se a coluna de label foi especificada
        if ( (self.label_column >= 0) and (self.label_column in cols_to_drop) ) or ( (self.label_column < 0) and ((self.label_column + n) in cols_to_drop) ):
            raise ValueError("Não é permitido remover a coluna de label")
        
        # Calcula as colunas restantes
        remaining_cols = [i for i in range(n) if i not in cols_to_drop]

        # Atualiza a posição do label = label atual menos a quantidade de atributos anteriores que foram removidos
        new_label_column = self.label_column - sum( 1 for col in cols_to_drop if col < self.label_column )

        # Atualiza a lista de nomes, se estiver definida
        remaining_label_column = None

        if self.column_names:
            remaining_label_column = [label for i, label in enumerate(self.column_names) if i not in cols_to_drop]

        # Atualiza o DataFrame para as colunas restantes
        new_data = new_data.iloc[:, remaining_cols]
        
        # Retorna uma nova instância para Dataset com os atributos restantes
        return self.__class__( new_data, label_column = new_label_column, column_names = remaining_label_column, _label_categories = self._label_categories )

    def normalize( self ) -> Self:
        """ Retorna o conjunto de dados com atributos normalizados em [-1, +1] """
        m, n, k = self.shape                        # Dimensões do Dataset
        new_data = self.data.copy( deep = True )    # Cópia do conjunto de dados

        # Percorre os índices dos atributos
        for idx in range( n ):
            # Obtém o subconjunto do atributo de índice idx
            column = new_data.iloc[:, idx]

            # Obtém os valores de mínimo e máximo do subconjunto
            _min = np.min( column )
            _max = np.max( column )
            
            # Subconjunto normalizado
            new_column = 2 * (column - _min) / (_max - _min) - 1

            # Ataliza o conjunto de dados com o subconjunto normalizado
            new_data.iloc[:, idx] = new_column
        
        # Retorna uma nova instância normalizada
        return self.__class__( new_data, label_column = self.label_column, column_names = self.column_names, _label_categories = self._label_categories )

    def ensure_integer_labels( self ) -> Self:
        """ Converte os valores dos rótulos para inteiros. """

        # Cópia do conjunto de dados
        new_data = self.data.copy( deep = True )

        # Obtém o subconjunto do rótulo
        label_column = new_data.iloc[:, self.label_column]

        # Verifica se já é inteiro (nada a fazer)
        if pd.api.types.is_integer_dtype( label_column ):
            return self

        # Usa factorize para converter os rótulos em inteiros
        encoded_labels, uniques = self.data.iloc[:, self.label_column].factorize()

        # Atribui a coluna convertida para 
        new_data.iloc[:, self.label_column] = encoded_labels

        # Dicionário com o mapeamento inteiro-rótulo
        self._label_categories = dict( enumerate(uniques) )

        # Retorna uma nova instância normalizada
        return self.__class__( new_data, label_column = self.label_column, column_names = self.column_names, _label_categories = self._label_categories )
    
    @classmethod
    def from_file( cls, filepath : str, *, comment_marker : str = "#", missing_marker : str = "?", label_column : int = -1, column_names : Optional[list[str]] = None ) -> None:
        """
        Inicializa a classe a partir de um arquivo CSV.

        Args:
            filepath: Caminho para o arquivo CSV.
            comment_marker: Marcador para linhas de comentário (serão ignoradas).
            missing_marker: Marcador para elementos faltantes (substituídos por NaN).
            Posição da coluna de rótulo (por padrão é -1, última coluna).
            column_names: Lista opcional com o nome das colunas.
        """

        # Lê o arquivo CSV (Não existe linhas de header no arquivo)
        data = pd.read_csv( filepath, header=None, comment=comment_marker, na_values=missing_marker )

        # Remover elementos faltantes
        data = data.dropna()

        # Retorna uma chamada para o construtor normal da classe
        return cls( data, label_column = label_column, column_names = column_names )
    
    @property
    def X( self ) -> pd.DataFrame:
        """ Retorna as colunas de atributos """

        # Caso mais simples, a coluna de label está no final
        if self.label_column == -1:
            return self.data.iloc[:, :self.label_column]
        
        # Se a coluna de resultados estiver em uma posição diferente da coluna final, compõe o DataFrame com os atributos
        return pd.concat([
            self.data.iloc[:, :self.label_column],  
            self.data.iloc[:, self.label_column+1:]
        ], axis=1 )        
    
    @property
    def y( self ) -> pd.Series:
        """ Retorna a coluna de rótulo """

        return self.data.iloc[:, self.label_column]
    
    @property
    def shape( self ) -> tuple[int, int, int]:
        """
        Retorna as dimensões do Dataset

        Returns:
            Tupla[m, n, k] onde:
            m = número de instâncias
            n = número de atributos 
            k = número de classes
        """

        m, n = self.data.shape
        k = len( self.y.unique() )

        # A coluna de rótulo está contada em n
        return (m, n-1, k)
    
    def __len__( self ):
        """ Retorna o número de instâncias do conjunto de dados. """
        return len( self.data )
    
    def __repr__( self ):
        """ Retorna uma string representando o objeto. """
        m, n, k = self.shape
        return f"Dataset(instâncias={m}, features={n}, classes={k})"
    
    def __iter__( self ):
        """ Retorna o objeto iterável da classe """
        return self.data.itertuples()

In [120]:
path_dataset = r"datasets\iris.data"    # caminho do arquivo
column_names = ["sepal length", "sepal width", "petal length", "petal width", "class"] # Nome dos campos

data = Dataset.from_file( path_dataset, label_column=-1, column_names=column_names ).ensure_integer_labels().normalize()

In [121]:
data.data

,sepal length,sepal width,petal length,petal width,class
0,-0.555556,0.250000,-0.864407,-0.916667,0
1,-0.666667,-0.166667,-0.864407,-0.916667,0
2,-0.777778,0.000000,-0.898305,-0.916667,0
3,-0.833333,-0.083333,-0.830508,-0.916667,0
4,-0.611111,0.333333,-0.864407,-0.916667,0
...,...,...,...,...,...
145,0.333333,-0.166667,0.423729,0.833333,2
146,0.111111,-0.583333,0.355932,0.500000,2
147,0.222222,-0.166667,0.423729,0.583333,2
148,0.055556,0.166667,0.491525,0.833333,2
